# न्यूरल नेटवर्क आर्किटेक्चर सर्च (एनएएस) अभ्यास

## कार्य: एक सरल NAS एल्गोरिदम बनाएं

ग्रिड खोज: एल्गोरिथम विचार, सभी संभावित संयोजनों पर ग्रिड खोज करें और सबसे उपयुक्त संयोजन का चयन करें

पायथन फ़ंक्शन लाइब्रेरी लोड करें

In [ ]:
import random
import numpy as np
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
import torch.nn.functional as F
from torchvision import datasets


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

# डेटा सेट लोड करें और प्रशिक्षण प्रक्रिया को परिभाषित करें

In [ ]:
# सामान्यीकरण सेट करें
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# डेटा सेट प्राप्त करें
train_dataset = datasets.MNIST(root='../ch02/data/mnist', train=True, download=True, transform=transform)  
test_dataset = datasets.MNIST(root='../ch02/data/mnist', train=False, download=True, transform=transform)  # train=True训练集，=False测试集

# डेटालोडर सेट करें
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def train(
  model: nn.Module,
  dataloader: DataLoader,
  criterion: nn.Module,
  optimizer: Optimizer,
# अनुसूचक: LambdaLR,
  callbacks = None
) -> None:
  model.train()

  for inputs, targets in tqdm(dataloader, desc='train', leave=False):
# डेटा को सीपीयू से जीपीयू में ले जाएं
# इनपुट = इनपुट.टू('एमपीएस')
# लक्ष्य = लक्ष्य.से('एमपीएस')

# ग्रेडिएंट्स को रीसेट करें (अंतिम पुनरावृत्ति से)
    optimizer.zero_grad()

#आगे का अनुमान
    outputs = model(inputs.cuda()).cpu()
    loss = criterion(outputs, targets)

#पिछड़ा प्रचार
    loss.backward()

# ऑप्टिमाइज़र और एलआर शेड्यूलर को अपडेट करें
    optimizer.step()
# शेड्यूलर.स्टेप()

    if callbacks is not None:
        for callback in callbacks:
            callback()

In [ ]:
@torch.inference_mode()
def evaluate(
  model: nn.Module,
  dataloader: DataLoader,
  extra_preprocess = None
) -> float:
  model.eval()

  num_samples = 0
  num_correct = 0

  for inputs, targets in tqdm(dataloader, desc="eval", leave=False):
# डेटा को सीपीयू से जीपीयू में ले जाएं
# इनपुट = इनपुट.टू('एमपीएस')
    if extra_preprocess is not None:
        for preprocess in extra_preprocess:
            inputs = preprocess(inputs)

# लक्ष्य = लक्ष्य.से('एमपीएस')

#अनुमान
    outputs = model(inputs.cuda()).cpu()

# लॉग को क्लास इंडेक्स में बदलें
    outputs = outputs.argmax(dim=1)

# मेट्रिक्स अपडेट करें
    num_samples += targets.size(0)
    num_correct += (outputs == targets).sum()

  return (num_correct / num_samples * 100).item()

In [ ]:
def get_model_flops(model, inputs):
    num_macs = profile_macs(model, inputs)
    return num_macs

def get_model_size(model: nn.Module, data_width=32):
    """
    calculate the model size in bits
    :param data_width: #bits per element
    """
    num_elements = 0
    for param in model.parameters():
        num_elements += param.numel()
    return num_elements * data_width

Byte = 8
KiB = 1024 * Byte
MiB = 1024 * KiB
GiB = 1024 * MiB

# तंत्रिका नेटवर्क वास्तुकला खोज

## कार्य, दिए गए सीमित मॉडल आकार के तहत सर्वोत्तम परिणाम ढूंढें

खोज विधि: ग्रिड खोज

स्थान खोजें: LeNet पर, विभिन्न ट्रैवर्सल स्थितियों को परिभाषित करें, उन्हें संयोजित करें, और फिर क्रम में मॉडल आकार को प्रशिक्षित और सत्यापित करें

आउटपुट: अंत में उपयुक्त तंत्रिका नेटवर्क संरचना को आउटपुट करें और इसे सहेजें

In [ ]:
# LeNet नेटवर्क को परिभाषित करें
class LeNet(nn.Module):
    def __init__(self, conv1_channel=6, conv1_kernel=5, conv2_channel=16, conv2_kernel=5, fc1_size=128, fc2_size=84, num_classes=10):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=conv1_channel, kernel_size=conv1_kernel) 
# 1 x 28 x 28 -> conv1_channel x (28-conv1_kernel+1) x (28-conv1_kernel+1)
# conv1_channel x (28-conv1_kernel+1) x (28-conv1_kernel+1) -> conv1_channel x (28-conv1_kernel+1)//2 x (28-conv1_kernel+1)//2
        self.conv2 = nn.Conv2d(in_channels=conv1_channel, out_channels=conv2_channel, kernel_size=conv2_kernel) 
# conv1_channel x (28-conv1_kernel+1)//2 x (28-conv1_kernel+1)//2 -> conv2_channel x ((28-conv1_kernel+1)//2- conv2_kernel + 1) x ((28-conv1_kernel +1)//2- conv2_कर्नेल + 1)
# conv2_channel x ((28-conv1_kernel+1)//2- conv2_kernel + 1) x ((28-conv1_kernel+1)//2- conv2_kernel + 1) -> conv2_channel x ((28-conv1_kernel+1)// 2- conv2_kernel + 1)//2 x ((28-conv1_kernel+1)//2- conv2_kernel + 1)//2

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        linear_input_dim = conv2_channel * (((28-conv1_kernel+1)//2-conv2_kernel+1)//2)**2 

        self.fc1 = nn.Linear(in_features=linear_input_dim, out_features=fc1_size)
        self.fc2 = nn.Linear(in_features=fc1_size, out_features=fc2_size)
        self.fc3 = nn.Linear(in_features=fc2_size, out_features=num_classes)

    def forward(self, x):
        x = self.maxpool(F.relu(self.conv1(x)))
        x = self.maxpool(F.relu(self.conv2(x)))
        
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# मॉडल = LeNet().to(डिवाइस=डिवाइस)

In [ ]:
# खोज स्थान को परिभाषित करें
conv1_channel_list = [3, 6, 9]
conv1_kernel_list = [3, 5, 7]
conv2_channel_list = [12, 16, 20]
conv2_kernel_list = [3, 5, 7]
fc1_size_list = [64, 128, 256]
fc2_size_list = [32, 84, 120]

# मॉडल प्रशिक्षण पैरामीटर कॉन्फ़िगरेशन
lr = 0.01
momentum = 0.5
num_epoch = 5
criterion = nn.CrossEntropyLoss()  # 交叉熵损失
limitation_model_size = 0.05 # 限制模型大小

# सर्वश्रेष्ठ वैश्विक रिकॉर्ड दर्ज करें
best_model = None
best_model_info = ""
overall_best_accuracy = 0

## खोजना शुरू करें
print("Searning")
for conv1_channel in conv1_channel_list:
    for conv1_kernel in conv1_kernel_list:
        for conv2_channel in conv2_channel_list:
            for conv2_kernel in conv2_kernel_list:
                for fc1_size in fc1_size_list:
                    for fc2_size in fc2_size_list:
                        model = LeNet(conv1_channel, conv1_kernel, conv2_channel, conv2_kernel, fc1_size, fc2_size).to(device=device)
                        fp32_model_size = get_model_size(model)
                        if fp32_model_size/MiB < limitation_model_size:
                            print(f"conv1_channel:{conv1_channel}, conv1_kenle:{conv1_kernel}, conv2_channel:{conv2_channel}, conv2_kenle:{conv2_kernel}, "\
                              f"fc1_size:{fc1_size}, fc2_size:{fc2_size}")
                            print(f"model has size={fp32_model_size/MiB:.2f} MiB")
                            optimizer = torch.optim.SGD(model.parameters(),  lr=lr, momentum=momentum)  # lr学习率，momentum冲量
                            best_accuracy = 0 
                            best_checkpoint = dict()
                            for epoch in range(num_epoch):
                                train(model, train_loader, criterion, optimizer)
                                accuracy = evaluate(model, test_loader)
                                is_best = accuracy > best_accuracy
                                if is_best:
                                    best_checkpoint['state_dict'] = copy.deepcopy(model.state_dict())
                                    best_accuracy = accuracy
                                print(f'Epoch{epoch+1:>2d} Accuracy {accuracy:.2f}% / Best Accuracy: {best_accuracy:.2f}%')
                            model.load_state_dict(best_checkpoint['state_dict'])
                            model_accuracy = evaluate(model, test_loader)
                            print(f"Model has accuracy={model_accuracy:.2f}%")
# वैश्विक रिकॉर्ड
                            if model_accuracy > overall_best_accuracy:
                                overall_best_accuracy = model_accuracy
                                best_model = model
                                best_model_info = f"conv1_channel:{conv1_channel}, conv1_kenle:{conv1_kernel}, conv2_channel:{conv2_channel}, conv2_kenle:{conv2_kernel}, "\
                              f"fc1_size:{fc1_size}, fc2_size:{fc2_size}"

In [ ]:
# सर्वोत्तम वैश्विक परिणाम आउटपुट करें
print("the result of searching is "+ best_model_info)
print(f"the accumulate of best model is:{overall_best_accuracy:.2f}%")
# प्रिंट(best_model)
